In [34]:
# Importing library
import pandas as pd
import os
import glob
import warnings
warnings.filterwarnings('ignore')
import math 
import mysql.connector as msql

In [35]:
# Displaying all columns and rows
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [36]:
# Taking files from define path which are in csv format
path = r'C:\Users\HP\OneDrive - VigyanShaala\02 Products  Initiatives\01 Kalpana\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Data Pipeline\Graphy Webscraping Code\Source File'

# Change the current working directory to the specified path
os.chdir(path)

# Use glob to get a list of file paths for all CSV files in the current directory
file_paths = glob.glob('*.csv')

In [37]:
# Create an empty list to store the keywords
keywords = []

# Create an empty dictionary to store the dataframes
dataframes = {}

# Iterate through each file path in the list of file paths
for file in file_paths:
    # Find the starting position of the keyword by searching for 'Assignment_'
    start = file.find('Assignment_') + len('Assignment_')
    
    # Find the ending position of the keyword by searching for '.csv'
    end = file.rfind('.csv')
    
    # Extract the keyword from the file path, removing leading/trailing spaces, and replacing spaces with underscores
    keyword = file[start:end].strip().replace(' ', '_')
    
    # Add the extracted keyword to the list of keywords
    keywords.append(keyword)
    
    # Read the CSV file and store it in a dataframe with the keyword as the key in the dataframes dictionary
    dataframes[keyword] = pd.read_csv(file)


In [38]:
# List of keyword of our csv file present in our folder
keywords

['Assignment_Long_term_dreams_and_short_term_goals',
 'Assignment_Practice',
 'Assignment_Preparing_for_PG_Masters',
 'Assignment_SMART_goal',
 'Assignment_SWOT']

In [39]:
# Iterate through each keyword in the list of keywords
for keyword in keywords:
    # Assign the corresponding dataframe to a variable with the same name as the keyword
    # This is done by using the globals() function to create a new variable in the global namespace
    globals()[keyword] = dataframes[keyword]

    # We have use golabl function to create dataframe of keyword so it should take name of dataframe as keyword. Like Swot will be name of datafarme.

In [40]:
# Reading Kalpana SHE for STEM Incubator file from Source files

directory_path =(r"C:\Users\HP\OneDrive - VigyanShaala\02 Products  Initiatives\01 Kalpana\05 Kalpana M&E\00 DBMS 1.0\Kalpana\TSWRDC Code\Source\Student Info")

csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

for file in csv_files:
    file_path = os.path.join(directory_path, file)
    Kalpana = pd.read_csv(file_path,usecols=['Email'])
    print(f"Data from {file}:")

In [8]:
import os
import pandas as pd

directory_path = r"C:\Users\HP\OneDrive - VigyanShaala\02 Products  Initiatives\01 Kalpana\05 Kalpana M&E\00 DBMS 1.0\Kalpana\TSWRDC Code\Source\Student Info"

excel_files = [file for file in os.listdir(directory_path) if file.endswith('.xlsx')]

for file in excel_files:
    file_path = os.path.join(directory_path, file)
    Kalpana = pd.read_excel(file_path, usecols=['Email'])
    clg = pd.read_excel(file_path)
    print(f"Data from {file}:")


Data from Final_Student Nominations.xlsx:


In [41]:
clg.head()

,Sno.,Name of the college,Student Name,Email,Currently pursuing year/semester,Subject Area
0,1.0,TSWRDC Siddipet,K.POOJA,kasapooja61@gmail.com,"2nd year, 4thSemester",BZC
1,2.0,TSWRDC Siddipet,P.DIVYA,palanchadeepiu@gmail.com,"2nd year,4th semester",BZC
2,3.0,TSWRDC Siddipet,B.MAMATHA,bgmamatha2022@gmail.com,"2nd year ,4th semester",BZC
3,4.0,TSWRDC Siddipet,T.SHRAVANI,ravitheddu@gmail.com,2nd year 4th semester,BZC
4,5.0,TSWRDC Siddipet,CH.JAYASRI,chatlapallyjayasri@gmail.com,"2nd year, 4th semester",BZC


In [42]:

# Create an empty dataframe to store the final results
final_df = pd.DataFrame()

# Create a dictionary to store comments for each keyword
comments_dict = {}

# Iterate through each keyword in the list of keywords
for keyword in keywords:
    # Filter the columns based on specific conditions...
    filtered_columns = [col for col in globals()[keyword] if ('message' in col) or ('email' in col) or (col == 'status')]

    new_df = globals()[keyword][filtered_columns]
    
    message_columns = [col for col in new_df.columns if 'message' in col]
    new_df[message_columns] = new_df[message_columns].fillna('')
    
    # Create a new 'Comment' column by joining the values of the 'message' columns for each row
    comment_col_name = 'Comment_' + keyword
    new_df[comment_col_name] = new_df[message_columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
    
    # Store the comments column name in the dictionary
    comments_dict[keyword] = comment_col_name
    
    # Modify the 'Email' column by removing a specific prefix from the values
    new_df['Email'] = new_df['user/email'].str.replace('vigyanshaalainternational1617-', '')
    
    # Rename the 'status' column to the keyword
    new_df = new_df.rename(columns={'status': keyword})
    
    # Select the desired columns for the new dataframe
    selected_columns = ['Email', keyword, comment_col_name]
    new_df = new_df[selected_columns]
    
    # If the final dataframe is empty, assign it as the new_df
    if final_df.empty:
        final_df = new_df.copy()
    # If the final dataframe is not empty, merge the new_df with the existing final dataframe using an outer join
    else:
        final_df = pd.merge(final_df, new_df, on='Email', how='outer', suffixes=('', f'_{keyword}'))

# Fill NaN values in the final dataframe
final_df = final_df.fillna('')

# Now, final_df contains all unique emails from all keywords


In [43]:
final_df.shape

(1400, 11)

In [44]:
final_df.columns

Index(['Email', 'Assignment_Long_term_dreams_and_short_term_goals',
       'Comment_Assignment_Long_term_dreams_and_short_term_goals',
       'Assignment_Practice', 'Comment_Assignment_Practice',
       'Assignment_Preparing_for_PG_Masters',
       'Comment_Assignment_Preparing_for_PG_Masters', 'Assignment_SMART_goal',
       'Comment_Assignment_SMART_goal', 'Assignment_SWOT',
       'Comment_Assignment_SWOT'],
      dtype='object')

In [45]:
# Create a list of columns to assign, excluding 'Email' and columns starting with 'Comment'
assign = [col for col in final_df.columns if col != 'Email' and not col.startswith('Comment')]


In [46]:
# Remove leading/trailing spaces in the selected columns
final_df[assign] = final_df[assign].applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [47]:
# To get unique values of dataframe to see what columns it consist to give 'marks'
final_df['Assignment_Practice'].unique()

array(['', 'reviewed', 'rejected'], dtype=object)

In [48]:
# To get unique values of dataframe to see what columns it consist to give 'marks'
final_df['Assignment_Long_term_dreams_and_short_term_goals'].unique()

array(['reviewed', 'rejected', 'under review', ''], dtype=object)

In [49]:
final_df.head()

,Email,Assignment_Long_term_dreams_and_short_term_goals,Comment_Assignment_Long_term_dreams_and_short_term_goals,Assignment_Practice,Comment_Assignment_Practice,Assignment_Preparing_for_PG_Masters,Comment_Assignment_Preparing_for_PG_Masters,Assignment_SMART_goal,Comment_Assignment_SMART_goal,Assignment_SWOT,Comment_Assignment_SWOT
0,vanjaresubhash015@gmail.com,reviewed,Successfully Submitted You have uploaded in th...,,,,,under review,Successfully Submitted You have to watch the r...,,
1,ravalikakonganti2004@gmail.com,reviewed,Successfully Submitted Good job. You can add m...,,,,,under review,Successfully Submitted You have understood the...,under review,Successfully Submitted
2,onapakalaarchana15@gmail.com,reviewed,Successfully Submitted Good job. You can add m...,,,,,under review,Successfully Submitted Good job. You have unde...,under review,Successfully Submitted
3,tollakavyakavya@gmail.com,rejected,Successfully Submitted You have submitted blan...,,,,,rejected,Successfully Submitted You have to watch the r...,,
4,doretisowmyareddy@gmail.com,rejected,Successfully Submitted You have submitted blan...,,,,,rejected,Successfully Submitted You have to watch the r...,,


### If you get new unique values in our column then add marks to it. such example "Pass" then at Please add pass in below dict and give marks to it like replace_dict = {'under review': 30, 'reviewed': 30,'rejected': 80, 'Pass': 100}

# Below you can give marks for Assignment 👇⬇️

In [50]:
replace_dict = {'under review': 5, 'reviewed': 30,'rejected': 15,'': 0}


In [18]:
#replace_dict = {'under review': 'Under review', 'reviewed': 'Accepted','rejected': 'Rejected','': "Not Submitted"}

# 👆☝️


In [51]:
# Replace values in selected columns
final_df[assign] = final_df[assign].replace(replace_dict)

In [52]:
# Convert the selected columns to float
final_df[assign] = final_df[assign].astype(float)

In [53]:
# Filling NaN Values with zero
final_df[assign]=final_df[assign].fillna(0)

In [54]:
# Calculate the sum of selected columns
sum_of_scores = final_df[assign].sum(axis=1)

# Count the number of selected columns
num_of_columns = len(assign)

# Calculate the average score
final_df['Assignment_Score'] = sum_of_scores / num_of_columns


In [55]:
num_of_columns

5

In [56]:
# Rounding Off values of Assignment score to two digits
final_df['Assignment_Score']=final_df['Assignment_Score'].round(2)

In [57]:
# Filling NaN Values with empty
final_df=final_df.fillna('')

In [58]:
final_df = final_df.replace('Successfully Submitted ', '# ', regex=True)


In [59]:
final_df.shape

(1400, 12)

In [60]:
# Checking our data
final_df.head(50)

,Email,Assignment_Long_term_dreams_and_short_term_goals,Comment_Assignment_Long_term_dreams_and_short_term_goals,Assignment_Practice,Comment_Assignment_Practice,Assignment_Preparing_for_PG_Masters,Comment_Assignment_Preparing_for_PG_Masters,Assignment_SMART_goal,Comment_Assignment_SMART_goal,Assignment_SWOT,Comment_Assignment_SWOT,Assignment_Score
0,vanjaresubhash015@gmail.com,30.0,# You have uploaded in the wrong module. Pleas...,0.0,,0.0,,5.0,# You have to watch the recording. Talk to you...,0.0,,7.0
1,ravalikakonganti2004@gmail.com,30.0,# Good job. You can add more specifics of whic...,0.0,,0.0,,5.0,# You have understood the concept. You have to...,5.0,#,8.0
2,onapakalaarchana15@gmail.com,30.0,# Good job. You can add more specifics of whic...,0.0,,0.0,,5.0,# Good job. You have understood the concept. Y...,5.0,#,8.0
3,tollakavyakavya@gmail.com,15.0,# You have submitted blank page. Please submit...,0.0,,0.0,,15.0,# You have to watch the recording. Talk to you...,0.0,,6.0
4,doretisowmyareddy@gmail.com,15.0,# You have submitted blank page. Please submit...,0.0,,0.0,,15.0,# You have to watch the recording. Talk to you...,0.0,,6.0
5,battudeepthinayak153@gmail.com,30.0,# Good job. You can add more specifics of whic...,0.0,,0.0,,15.0,# You have to watch the recording. Talk to you...,0.0,,9.0
6,katkuripooja123@gmail.com,30.0,# Good job. You can add more specifics of whic...,0.0,,0.0,,30.0,# Good job. You have understood the concept. Y...,0.0,,12.0
7,aishwaryakada046@gmail.com,30.0,# Good job. You can add more specifics of whic...,0.0,,0.0,,30.0,# Good job. You have understood the concept. Y...,0.0,,12.0
8,chiluwarijansi@gmail.com,15.0,# You have submitted blank page. Please submit...,0.0,,0.0,,15.0,"# S: The goal needs to be very specific, to t...",0.0,,6.0
9,begarishalini144@gmail.com,30.0,# Good job. You can add more specifics of whic...,0.0,,0.0,,15.0,"# S: The goal needs to be very specific, to t...",0.0,,9.0


In [61]:
# Iterate through all columns in the dataframe
for col in final_df.columns:
    # Check if column name contains '.'
    if '.' in col:
        # Replace '.' with '_' and remove ' ' only where '.' is present in the column name
        new_col = col.replace('.', '_').replace(' ', '') if '.' in col else col
        # Rename the column
        df.rename(columns={col: new_col}, inplace=True)


In [62]:
# Cheaking the columns present in the dataframe
final_df.columns

Index(['Email', 'Assignment_Long_term_dreams_and_short_term_goals',
       'Comment_Assignment_Long_term_dreams_and_short_term_goals',
       'Assignment_Practice', 'Comment_Assignment_Practice',
       'Assignment_Preparing_for_PG_Masters',
       'Comment_Assignment_Preparing_for_PG_Masters', 'Assignment_SMART_goal',
       'Comment_Assignment_SMART_goal', 'Assignment_SWOT',
       'Comment_Assignment_SWOT', 'Assignment_Score'],
      dtype='object')

In [63]:
import re
# Function to clean email addresses
def clean_email(email):
    # Convert to lowercase and remove extra spaces
    cleaned_email = email.lower().strip()
    # Remove patterns like ".com.1"
    cleaned_email = re.sub(r'\.com\.\d+', '.com', cleaned_email)
    return cleaned_email

# Apply the function to the 'email' column
final_df['Email'] = final_df['Email'].apply(clean_email)

In [64]:
import re
# Function to clean email addresses
def clean_email(email):
    # Convert to lowercase and remove extra spaces
    cleaned_email = email.lower().strip()
    # Remove patterns like ".com.1"
    cleaned_email = re.sub(r'\.com\.\d+', '.com', cleaned_email)
    return cleaned_email

# Apply the function to the 'email' column
Kalpana['Email'] = Kalpana['Email'].apply(clean_email)

In [65]:
import re
# Function to clean email addresses
def clean_email(email):
    # Convert to lowercase and remove extra spaces
    cleaned_email = email.lower().strip()
    # Remove patterns like ".com.1"
    cleaned_email = re.sub(r'\.com\.\d+', '.com', cleaned_email)
    return cleaned_email

# Apply the function to the 'email' column
clg['Email'] = clg['Email'].apply(clean_email)

In [66]:
final_df = pd.merge(final_df, Kalpana, on='Email', how='inner')


In [30]:
clg_df = pd.merge(clg,final_df, on='Email', how='inner')

In [31]:
clg_df.shape

(1362, 16)

In [67]:
final_df.head()

,Email,Assignment_Long_term_dreams_and_short_term_goals,Comment_Assignment_Long_term_dreams_and_short_term_goals,Assignment_Practice,Comment_Assignment_Practice,Assignment_Preparing_for_PG_Masters,Comment_Assignment_Preparing_for_PG_Masters,Assignment_SMART_goal,Comment_Assignment_SMART_goal,Assignment_SWOT,Comment_Assignment_SWOT,Assignment_Score
0,ravalikakonganti2004@gmail.com,30.0,# Good job. You can add more specifics of whic...,0.0,,0.0,,5.0,# You have understood the concept. You have to...,5.0,#,8.0
1,onapakalaarchana15@gmail.com,30.0,# Good job. You can add more specifics of whic...,0.0,,0.0,,5.0,# Good job. You have understood the concept. Y...,5.0,#,8.0
2,tollakavyakavya@gmail.com,15.0,# You have submitted blank page. Please submit...,0.0,,0.0,,15.0,# You have to watch the recording. Talk to you...,0.0,,6.0
3,doretisowmyareddy@gmail.com,15.0,# You have submitted blank page. Please submit...,0.0,,0.0,,15.0,# You have to watch the recording. Talk to you...,0.0,,6.0
4,battudeepthinayak153@gmail.com,30.0,# Good job. You can add more specifics of whic...,0.0,,0.0,,15.0,# You have to watch the recording. Talk to you...,0.0,,9.0


In [32]:
clg_df.drop(columns=['Sno.', 'Currently pursuing year/semester', 'Subject Area'], inplace=True)



In [68]:
#clg_df.to_csv(r"Student_Assignment_14_mar.csv",mode='w',index=False)
final_df.to_csv(r"Assignment_14_mar.csv",mode='w',index=False)

### The step remaing here is renaming i.e while putting to our mysql

In [69]:
# Get the list of column names in the final_df
column_names = final_df.columns

# Iterate through the column names and rename columns starting with "Comment"
for col in column_names:
    if col.startswith("Comment"):
        new_col_name = "Comment"
        final_df = final_df.rename(columns={col: new_col_name})



In [70]:
# Checking the renaming column name
final_df.columns

Index(['Email', 'Acing_your_CV_and_Resume', 'Comment',
       'Career_Action_Plan_(CAP)', 'Comment', 'Career_Exploration', 'Comment',
       'Goal_Setting', 'Comment', 'Making_the_most_of_LinkedIn', 'Comment',
       'Planning_your_Applications_–_Higher_Education', 'Comment',
       'RIASEC_personality_test', 'Comment', 'Searching_&_Securing_Internship',
       'Comment', 'SMART_goal', 'Comment', 'SWOT', 'Comment',
       'Assignment_Score'],
      dtype='object')

# Below insted of renaming name if at staring only we have taken the assignment name as we want mention in txt in source file we will not need to do below step 👇⬇️

In [71]:
# Renaming the column to put in datafarme 
new_names = {
    'SWOT': 'WK_1_SWOT_Analysis',
    'Career_Action_Plan_(CAP)': 'WK_3_Career_Action_Plan',
    'Critical_thinking': 'WK_6_Critical_thinking',
    'Masteclass_1_-_Career_Exploration': 'MC_Career_Exploration',
    'Masterclass_-_Academic_CV_and_Industry_Resume': 'MC_CV_Resume',
    'Networking__Build_a_World-Class_LinkedIn_Profile': 'MC_LinkedIn_Profile',
    'Planning_&_Applying_for_Masters_in_India_&_Abroad': 'MC_Planning_Masters',
    'RIASEC_personality_test': 'WK_2_RIASEC',
    'SMART_goal': 'WK_2_SMART_goal'
}

for col in final_df.columns:
    if col in new_names:
        final_df.rename(columns={col: new_names[col]}, inplace=True)


In [72]:
# Checking the renaming column name
final_df.columns

Index(['Email', 'Acing_your_CV_and_Resume', 'Comment',
       'WK_3_Career_Action_Plan', 'Comment', 'Career_Exploration', 'Comment',
       'Goal_Setting', 'Comment', 'Making_the_most_of_LinkedIn', 'Comment',
       'Planning_your_Applications_–_Higher_Education', 'Comment',
       'WK_2_RIASEC', 'Comment', 'Searching_&_Securing_Internship', 'Comment',
       'WK_2_SMART_goal', 'Comment', 'WK_1_SWOT_Analysis', 'Comment',
       'Assignment_Score'],
      dtype='object')

In [73]:
# Renaming the 'comment' in our given format
cols = final_df.columns.tolist()
new_cols = []
prev_col = ''

for col in cols:
    if col == 'Comment':
        new_cols.append(f'Comments_{prev_col}')
    else:
        new_cols.append(col)
    prev_col = col

final_df.columns = new_cols


In [74]:
# Checking the renaming column name
final_df.columns

Index(['Email', 'Acing_your_CV_and_Resume',
       'Comments_Acing_your_CV_and_Resume', 'WK_3_Career_Action_Plan',
       'Comments_WK_3_Career_Action_Plan', 'Career_Exploration',
       'Comments_Career_Exploration', 'Goal_Setting', 'Comments_Goal_Setting',
       'Making_the_most_of_LinkedIn', 'Comments_Making_the_most_of_LinkedIn',
       'Planning_your_Applications_–_Higher_Education',
       'Comments_Planning_your_Applications_–_Higher_Education', 'WK_2_RIASEC',
       'Comments_WK_2_RIASEC', 'Searching_&_Securing_Internship',
       'Comments_Searching_&_Securing_Internship', 'WK_2_SMART_goal',
       'Comments_WK_2_SMART_goal', 'WK_1_SWOT_Analysis',
       'Comments_WK_1_SWOT_Analysis', 'Assignment_Score'],
      dtype='object')

In [75]:
#final_df.to_csv('Career_Exploration_Withcomment.csv', mode='a',index=False)

In [2]:
final_df.to_csv(r"C:\Users\HP\OneDrive - VigyanShaala\02 Products  Initiatives\01 Kalpana\05 Kalpana M&E\00 DBMS 1.0\Kalpana\New_Assignment_Scrapping Method\Output\Webscraping_Assignment.csv",mode='w',index=False)

NameError: name 'final_df' is not defined

# Storing data on MySQL

In [1]:
# Connecting to MySQL Database
conn= msql.connect(host='localhost',user='root',password="VS@123",database="kalpana",auth_plugin='mysql_native_password')
#cursor =conn.cursor()

NameError: name 'msql' is not defined

☝️ REMOVE COMMENT OF ABOVE I.E REMOVE #

In [ ]:

# Get the existing columns in the database
cursor.execute("SHOW COLUMNS FROM kalpana.11_incubator_assignment_monitoring")
existing_columns = [col[0] for col in cursor.fetchall()]

# Define the column name before which the new column should be added
target_column = 'Assignment_Score'

# Check if any new columns exist in the dataframe but not in the database
new_columns = [col for col in final_df.columns if col not in existing_columns]
if new_columns:
    # Add new columns to the database before the target column
    for col in reversed(new_columns):
        if col not in existing_columns:
            # Get the index of the target column
            target_column_index = existing_columns.index(target_column)
            # Set the data type based on whether the column name starts with Comment
            data_type = "INT" if not col.startswith("Comment") else "TEXT"
            alter_query = f"ALTER TABLE kalpana.11_incubator_assignment_monitoring ADD COLUMN {col} {data_type} AFTER {existing_columns[target_column_index - 1]}"
            cursor.execute(alter_query)
            existing_columns.insert(target_column_index - 1, col)


In [ ]:
# Your existing code for inserting data into the database table
for i, row in final_df.iterrows():
    row = [None if isinstance(val, float) and math.isnan(val) else val for val in row] # replace "nan" values with None
    columns = ','.join(final_df.columns)
    placeholders = ','.join(['%s']*len(row))
    # Construct the INSERT query with ON DUPLICATE KEY UPDATE clause
    query = f"INSERT INTO kalpana.11_incubator_assignment_monitoring ({columns}) VALUES ({placeholders}) ON DUPLICATE KEY UPDATE "
    query += ", ".join([f"{col}=VALUES({col})" for col in final_df.columns if col != 'Email'])
    # Execute the query
    cursor.execute(query, tuple(row))


In [ ]:
conn.commit()